## MODELO DE CLUSTERIZACIÓN DE SOLICITUDES DE CRÉDITO (K-MEANS) PARA RIESGO DE CRÉDITO

#### NOTEBOOK: Clustering_model

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.functions import col, when, count, isnan, isnull, lit
from pyspark.sql.types import StructType, StructField, TimestampType, IntegerType, DoubleType, BooleanType
import requests
import os
from datetime import datetime

# Crear o obtener la sesión de Spark
spark = SparkSession.builder \
    .appName("ClusteringModel") \
    .getOrCreate()

print("Sesión de Spark activa:", spark)

# Cargar datos con manejo de errores
try:
    df = spark.read.format("delta").load("Tables/solicitudes_processed")
    print(f"Datos cargados exitosamente. Total de registros: {df.count()}")
except Exception as e:
    raise Exception(f"Error al cargar los datos: {str(e)}")

# Definir columnas de entrada
input_cols = ["edad", "ingresos_anuales", "puntaje_crediticio", "deuda_actual",
              "antiguedad_laboral", "historial_pagos_encoded", "estado_civil_encoded",
              "tipo_empleo_encoded", "numero_dependientes"]

# Análisis de valores nulos antes de la limpieza
print("\nAnálisis de valores nulos antes de la limpieza:")
for column in input_cols:
    null_count = df.filter(col(column).isNull() | isnan(col(column))).count()
    total = df.count()
    null_percentage = (null_count / total) * 100
    print(f"{column}: {null_count} nulos ({null_percentage:.2f}%)")

# Manejar valores nulos con estrategia más sofisticada
for col_name in input_cols:
    # Usar la mediana para variables numéricas
    if col_name in ["edad", "ingresos_anuales", "puntaje_crediticio", "deuda_actual", "antiguedad_laboral"]:
        median_value = df.approxQuantile(col_name, [0.5], 0.01)[0]
        df = df.fillna(median_value, subset=[col_name])
    else:
        # Usar 0 para variables categóricas codificadas
        df = df.fillna(0, subset=[col_name])

# Verificar que no queden nulos
null_check = df.select([count(when(col(c).isNull() | isnan(col(c)), c)).alias(c) for c in input_cols])
print("\nVerificación final de valores nulos:")
null_check.show()

# Dividir datos aleatoriamente para evitar sesgos
train_df, test_df = df.randomSplit([0.8, 0.2], seed=123)
print(f"Datos de entrenamiento: {train_df.count()} registros")
print(f"Datos de prueba: {test_df.count()} registros")

# Preparar características
assembler = VectorAssembler(
    inputCols=input_cols,
    outputCol="features",
    handleInvalid="skip"  # Manejo explícito de valores inválidos
)

# Transformar datos
features_df = assembler.transform(train_df).select("features")

# Método del codo para determinar k óptimo
print("Ejecutando método del codo...")
elbow_results = []
k_range = range(2, 11)  # Probar k de 2 a 10

try:
    for k in k_range:
        kmeans = KMeans(
            featuresCol="features",
            predictionCol="prediction",
            k=k,
            seed=123
        )
        model = kmeans.fit(features_df)
        cost = model.summary.trainingCost  # WSS
        elbow_results.append((k, cost))
        print(f"k={k}, Costo (WSS): {cost:.4f}")

    # Seleccionar k óptimo (heurística: menor disminución relativa en costo)
    cost_changes = [(k, elbow_results[i][1] / elbow_results[i-1][1]) for i, (k, _) in enumerate(elbow_results[1:], 1)]
    optimal_k = min(cost_changes, key=lambda x: x[1])[0] if cost_changes else 4
    print(f"\nNúmero óptimo de clústeres (k): {optimal_k}")

    # Crear DataFrame para resultados del método del codo con la nueva columna
    elbow_schema = StructType([
        StructField("fecha_entrenamiento", TimestampType(), False),
        StructField("k", IntegerType(), False),
        StructField("costo", DoubleType(), False),
        StructField("es_mejor_modelo", BooleanType(), False)
    ])

    # Preparar datos con la marca del mejor modelo
    current_time = datetime.now()
    elbow_data = [(
        current_time,
        k,
        float(cost),
        k == optimal_k  # True solo para el k óptimo
    ) for k, cost in elbow_results]

    elbow_df = spark.createDataFrame(elbow_data, schema=elbow_schema)

    # Guardar resultados del método del codo
    try:
        elbow_df.write.mode("append").format("delta").save("Tables/elbow_method_results")
        print("Resultados del método del codo guardados exitosamente")
        print(f"Se marcó k={optimal_k} como el mejor modelo")
    except Exception as e:
        print(f"Error al guardar resultados del método del codo: {e}")
        raise

except Exception as e:
    print(f"Error durante el método del codo: {e}")
    raise

# Entrenar modelo K-means final
kmeans = KMeans(
    featuresCol="features",
    predictionCol="prediction",
    k=optimal_k,
    seed=123
)

try:
    print("Iniciando entrenamiento del modelo K-means final...")
    kmeans_model = kmeans.fit(features_df)
    print("Modelo K-means entrenado exitosamente")

    # Evaluar modelo con datos de prueba
    test_features = assembler.transform(test_df).select("features")
    predictions = kmeans_model.transform(test_features)
    evaluator = ClusteringEvaluator(
        predictionCol="prediction",
        featuresCol="features",
        metricName="silhouette",
        distanceMeasure="squaredEuclidean"
    )
    silhouette_score = evaluator.evaluate(predictions)
    print(f"\nPuntaje de silueta: {silhouette_score:.4f}")

except Exception as e:
    raise Exception(f"Error durante el entrenamiento o evaluación: {str(e)}")

# Guardar modelo K-means
lakehouse_id = "a2c9f91b-6c69-4d1d-96f9-24b310914c2b"
model_path = f"Tables/Models/KMeansClustering"

try:
    print(f"\nGuardando modelo en: {model_path}")
    kmeans_model.write().overwrite().save(model_path)
    print(f"Modelo K-means guardado exitosamente en {model_path}")
except Exception as e:
    print(f"Error al guardar el modelo: {e}")
    # Intentar ruta alternativa
    try:
        alt_path = "./models/KMeansClustering"
        print(f"Intentando guardar en ruta alternativa: {alt_path}")
        kmeans_model.write().overwrite().save(alt_path)
        print(f"Modelo guardado en ruta alternativa: {alt_path}")
    except Exception as e2:
        print(f"Error al guardar en ruta alternativa: {e2}")
        raise

# Registrar métricas
metrics_schema = StructType([
    StructField("fecha_entrenamiento", TimestampType(), False),
    StructField("k", IntegerType(), False),
    StructField("silhouette_score", DoubleType(), False),
    StructField("costo_final", DoubleType(), False)
])

metrics_df = spark.createDataFrame([(
    datetime.now(),
    optimal_k,
    float(silhouette_score),
    float(kmeans_model.summary.trainingCost)
)], schema=metrics_schema)

try:
    metrics_df.write.mode("append").format("delta").save("Tables/kmeans_metrics")
    print("\nMétricas guardadas exitosamente")
except Exception as e:
    print(f"Error al guardar métricas: {e}")
    raise

# Notificar resultados a Discord
discord_webhook_url = os.getenv('DISCORD_WEBHOOK_URL', "XXXXXXXXXXXXX")

def crear_mensaje_exito(silhouette, k, costo):
    return {
        "embeds": [
            {
                "title": "✅ Reentrenamiento Exitoso - K-means Clustering",
                "description": "El modelo de clustering ha sido reentrenado exitosamente.",
                "color": 3066993,  # Verde
                "fields": [
                    {"name": "Número de clústeres (k)", "value": str(k), "inline": True},
                    {"name": "Puntaje de silueta", "value": f"{silhouette:.4f}", "inline": True},
                    {"name": "Costo final", "value": f"{costo:.4f}", "inline": True}
                ],
                "footer": {"text": "RiskApp - Sistema de Monitoreo"},
                "timestamp": datetime.now().isoformat()
            }
        ]
    }

def crear_mensaje_error(error):
    return {
        "embeds": [
            {
                "title": "❌ Error en el Reentrenamiento - K-means Clustering",
                "description": f"Se produjo un error durante el reentrenamiento: {error}",
                "color": 15158332,  # Rojo
                "footer": {"text": "RiskApp - Sistema de Monitoreo"},
                "timestamp": datetime.now().isoformat()
            }
        ]
    }

def enviar_notificacion(mensaje):
    try:
        response = requests.post(discord_webhook_url, json=mensaje)
        response.raise_for_status()
        print("\nNotificación enviada exitosamente a Discord")
    except Exception as e:
        print(f"Error al enviar notificación: {str(e)}")

try:
    mensaje = crear_mensaje_exito(
        silhouette=silhouette_score,
        k=optimal_k,
        costo=kmeans_model.summary.trainingCost
    )
    enviar_notificacion(mensaje)
except Exception as e:
    mensaje = crear_mensaje_error(str(e))
    enviar_notificacion(mensaje)
